## 6장 연습문제10

이 파일은 6장 연습문제10을 풀이한 내용입니다.

날짜 : 2025 - 09 - 24

저자 : 차일권

학번 : 202111467


10번 문제: 파이썬 언어에서 다음에 대해 조사하고 예제 코드를 실행하여 보라
• 1. 파이썬으로 싱글톤 패턴을 구현하는 방법들
• 2. 파이썬에서 멀티쓰레드 구현하는 기법
• 3. 멀티쓰레드에서 공유변수에 대한 Critical Section을 보호하는 기법


## 1. 파이썬으로 싱글톤 패턴을 구현하는 방법들

싱글톤 패턴은 클래스의 인스턴스가 하나만 생성되도록 보장하는 디자인 패턴입니다.


In [ ]:
# 방법 1: __new__ 메소드 사용
class Singleton1:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(Singleton1, cls).__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not hasattr(self, 'initialized'):
            self.initialized = True
            print("Singleton1 인스턴스가 생성되었습니다.")


# 방법 2: 데코레이터 사용
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Singleton2:
    def __init__(self):
        print("Singleton2 인스턴스가 생성되었습니다.")


# 방법 3: 메타클래스 사용
class SingletonMeta(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(SingletonMeta, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class Singleton3(metaclass=SingletonMeta):
    def __init__(self):
        print("Singleton3 인스턴스가 생성되었습니다.")


print("--- 싱글톤 패턴 테스트 ---")

# 방법 1 테스트
print("\n방법 1: __new__ 메소드 사용")
s1_1 = Singleton1()
s1_2 = Singleton1()
print(f"s1_1과 s1_2가 같은 객체인가? {s1_1 is s1_2}")

# 방법 2 테스트
print("\n방법 2: 데코레이터 사용")
s2_1 = Singleton2()
s2_2 = Singleton2()
print(f"s2_1과 s2_2가 같은 객체인가? {s2_1 is s2_2}")

# 방법 3 테스트
print("\n방법 3: 메타클래스 사용")
s3_1 = Singleton3()
s3_2 = Singleton3()
print(f"s3_1과 s3_2가 같은 객체인가? {s3_1 is s3_2}")


## 2. 파이썬에서 멀티쓰레드 구현하는 기법

파이썬에서는 threading 모듈을 사용하여 멀티쓰레드를 구현할 수 있습니다.


In [ ]:
import threading
import time
import random

# 공유 변수 (Critical Section 문제 발생 가능)
shared_counter = 0

def worker_function(worker_id: int, iterations: int):
    """작업자 함수: 공유 변수를 증가시키는 작업"""
    global shared_counter
    
    for i in range(iterations):
        # Critical Section: 공유 변수에 접근
        current_value = shared_counter
        time.sleep(random.uniform(0.001, 0.01))  # 시뮬레이션을 위한 지연
        shared_counter = current_value + 1
        print(f"Worker {worker_id}: 반복 {i+1}, 현재 카운터: {shared_counter}")

def thread_example():
    """멀티쓰레드 예제"""
    print("--- 멀티쓰레드 예제 (Critical Section 문제 발생) ---")
    
    # 쓰레드 생성
    threads = []
    num_threads = 3
    iterations_per_thread = 5
    
    start_time = time.time()
    
    for i in range(num_threads):
        thread = threading.Thread(target=worker_function, args=(i+1, iterations_per_thread))
        threads.append(thread)
        thread.start()
    
    # 모든 쓰레드가 완료될 때까지 대기
    for thread in threads:
        thread.join()
    
    end_time = time.time()
    
    print(f"\n실행 시간: {end_time - start_time:.2f}초")
    print(f"최종 카운터 값: {shared_counter}")
    print(f"예상 값: {num_threads * iterations_per_thread}")
    print(f"Race Condition 발생: {shared_counter != num_threads * iterations_per_thread}")

# 멀티쓰레드 실행
thread_example()


In [ ]:
# Critical Section 보호를 위한 Lock 사용
import threading
import time
import random

# 공유 변수와 Lock
protected_counter = 0
counter_lock = threading.Lock()

def protected_worker_function(worker_id: int, iterations: int):
    """Lock을 사용하여 Critical Section을 보호하는 작업자 함수"""
    global protected_counter
    
    for i in range(iterations):
        # Lock 획득 (Critical Section 진입)
        with counter_lock:  # 자동으로 lock.acquire()와 lock.release() 처리
            current_value = protected_counter
            time.sleep(random.uniform(0.001, 0.01))  # 시뮬레이션을 위한 지연
            protected_counter = current_value + 1
            print(f"Protected Worker {worker_id}: 반복 {i+1}, 현재 카운터: {protected_counter}")

def protected_thread_example():
    """Lock을 사용한 멀티쓰레드 예제"""
    print("\n--- Lock을 사용한 멀티쓰레드 예제 (Critical Section 보호) ---")
    
    # 공유 변수 초기화
    global protected_counter
    protected_counter = 0
    
    # 쓰레드 생성
    threads = []
    num_threads = 3
    iterations_per_thread = 5
    
    start_time = time.time()
    
    for i in range(num_threads):
        thread = threading.Thread(target=protected_worker_function, args=(i+1, iterations_per_thread))
        threads.append(thread)
        thread.start()
    
    # 모든 쓰레드가 완료될 때까지 대기
    for thread in threads:
        thread.join()
    
    end_time = time.time()
    
    print(f"\n실행 시간: {end_time - start_time:.2f}초")
    print(f"최종 카운터 값: {protected_counter}")
    print(f"예상 값: {num_threads * iterations_per_thread}")
    print(f"Race Condition 발생: {protected_counter != num_threads * iterations_per_thread}")

# RLock (Reentrant Lock) 예제
class BankAccount:
    def __init__(self, initial_balance: int = 0):
        self.balance = initial_balance
        self.lock = threading.RLock()  # 재진입 가능한 Lock
    
    def deposit(self, amount: int):
        with self.lock:
            self.balance += amount
            print(f"입금: {amount}, 잔액: {self.balance}")
    
    def withdraw(self, amount: int):
        with self.lock:
            if self.balance >= amount:
                self.balance -= amount
                print(f"출금: {amount}, 잔액: {self.balance}")
                return True
            else:
                print(f"출금 실패: 잔액 부족 (요청: {amount}, 잔액: {self.balance})")
                return False
    
    def transfer(self, other_account, amount: int):
        with self.lock:  # RLock이므로 같은 쓰레드에서 중첩 호출 가능
            if self.withdraw(amount):
                other_account.deposit(amount)
                print(f"이체 완료: {amount}")

def bank_example():
    """은행 계좌 예제 (RLock 사용)"""
    print("\n--- 은행 계좌 예제 (RLock 사용) ---")
    
    account1 = BankAccount(1000)
    account2 = BankAccount(500)
    
    def transfer_worker():
        for _ in range(3):
            account1.transfer(account2, 100)
            time.sleep(0.1)
    
    # 이체 작업을 수행하는 쓰레드들
    threads = []
    for i in range(2):
        thread = threading.Thread(target=transfer_worker)
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
    
    print(f"최종 계좌1 잔액: {account1.balance}")
    print(f"최종 계좌2 잔액: {account2.balance}")

# 실행
protected_thread_example()
bank_example()


In [ ]:
# 실행 결과
print("--- 싱글톤 패턴 테스트 ---")
s1_1 = Singleton1()
s1_2 = Singleton1()
print(f"s1_1과 s1_2가 같은 객체인가? {s1_1 is s1_2}")

s2_1 = Singleton2()
s2_2 = Singleton2()
print(f"s2_1과 s2_2가 같은 객체인가? {s2_1 is s2_2}")

s3_1 = Singleton3()
s3_2 = Singleton3()
print(f"s3_1과 s3_2가 같은 객체인가? {s3_1 is s3_2}")

# 멀티쓰레드 실행
thread_example()

# Critical Section 보호 실행
protected_thread_example()
bank_example()
